# Find my zipcodes

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

df = pd.read_csv('../data/King_County_House_prices_dataset.csv')

In [86]:
#getting all zipcodes
zips = df.zipcode.unique().astype(int)
zips
#seattle center zips
c_zips = [98101, 98102, 98103, 98104, 98105, 98106, 98107, 98108, 98109, 98112, 98115, 98116, 98117, 98118, 98119, 98121, 98122, 98125, 98126, 98133, 98134, 98136, 98144, 98146, 98154, 98164, 98174, 98177, 98178, 98195, 98199]

In [76]:
#getting the data for population per zip
#force string to prevent turning high numbers to float
pop_df = pd.read_csv('../data/population_by_zip.csv', dtype='string')
pop_df.drop('rank', inplace=True, axis=1)
#removing '.' from high numbers
pop_df.population = pop_df.population.str.replace('.','', regex=False)
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   zipcode     565 non-null    string
 1   population  565 non-null    string
dtypes: string(2)
memory usage: 9.0 KB


In [77]:
#turning obects into numerics
#getting all rows that can't be converted
non_ints = pop_df[['zipcode', 'population']][pop_df['zipcode'].str.isnumeric() == False]
pop_df= pop_df[['zipcode', 'population']][pop_df['zipcode'].str.isnumeric()]

In [78]:
non_ints.info()
pop_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 95 to 561
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   zipcode     16 non-null     string
 1   population  16 non-null     string
dtypes: string(2)
memory usage: 384.0 bytes


,zipcode,population
0,99301,79819
1,98052,70245
2,98012,67427
3,98682,60887
4,98208,58162


In [79]:
#removing the 'and' from non_ints TODO: find cleaner solution
non_ints = np.reshape([[x] for row in non_ints.values for x in (row[0].split('and')[0], row[0].split('and')[1], row[1])], (-1,3))
non_ints

array([['98685', '98201', '29792'],
       ['98597', '98075', '23996'],
       ['99003', '98237', '4410'],
       ['98862', '98568', '2461'],
       ['99110', '98592', '1839'],
       ['99139', '99180', '1301'],
       ['98836', '98933', '1051'],
       ['98336', '98276', '727'],
       ['98941', '98619', '539'],
       ['99140', '98050', '432'],
       ['98164', '98817', '177'],
       ['98255', '98614', '164'],
       ['98575', '98833', '139'],
       ['98330', '99151', '102'],
       ['98195', '98297', '48'],
       ['99144', '98353', '18']], dtype='<U5')

In [80]:
#feeding cleaned non_ints back to pop_df
for row in non_ints:
    new_rows = pd.DataFrame([[row[0], row[-1]], [row[1], row[-1]]], columns=['zipcode', 'population'])
    pop_df = pop_df.append(new_rows, ignore_index=True)

In [82]:
# getting avg population
pop_df[['zipcode', 'population']] = pop_df[['zipcode', 'population']].astype(int)
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581 entries, 0 to 580
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   zipcode     581 non-null    int64
 1   population  581 non-null    int64
dtypes: int64(2)
memory usage: 9.2 KB


In [90]:
#removing all zips that are not in our seattle data set
pop_df = pop_df[pop_df.zipcode.isin(zips)]
pop_df
avg_pop = pop_df.population.mean()
avg_pop

30491.0

In [93]:
#removing all zips from seattle center
osk_df = pop_df[pop_df.zipcode.isin(c_zips)]
osk_df.info()
#removing zips with above avg population
osk_df = osk_df[osk_df['population'] < avg_pop]
osk_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 6 to 177
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   zipcode     23 non-null     int64
 1   population  23 non-null     int64
dtypes: int64(2)
memory usage: 552.0 bytes
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 91 to 177
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   zipcode     14 non-null     int64
 1   population  14 non-null     int64
dtypes: int64(2)
memory usage: 336.0 bytes
